In [1]:
from kan import *
import pytz
import time
import datetime
from data_process import data_process_without_norm
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import copy
import pandas as pd

df = data_process_without_norm()
df = df.drop("entsoe",axis=1)
loc_tz = pytz.timezone('Europe/Zurich')
split_date_train_ = loc_tz.localize(datetime.datetime(2016,1,1,0,0,0,0))
split_date_train = loc_tz.localize(datetime.datetime(2016,3,1,0,0,0,0))
split_date_test = loc_tz.localize(datetime.datetime(2016,3,15,0,0,0,0))

df_train_ = df.loc[(split_date_train_ < df.index)]
df_train = df_train_.loc[df_train_.index <= split_date_train].copy()
_temp_df = df.loc[split_date_test > df.index]
df_test = _temp_df.loc[_temp_df.index > split_date_train].copy()

# Input standardization
scaler_input = StandardScaler()
# scaler_input = MinMaxScalerr()
_temp_scaled_input_data = scaler_input.fit_transform(df_train.iloc[:,1:])
x_train_input = _temp_scaled_input_data

#Output standardization
scaler_output = StandardScaler()
# scaler_input = MinMaxScaler()
_temp_scaled_output_data = scaler_output.fit_transform(np.array(df_train.iloc[:,0]).reshape(-1,1))
x_train_label = _temp_scaled_output_data

#Test set standardization
x_test_input = scaler_input.transform(df_test.iloc[:,1:])
x_test_label = scaler_output.transform(np.array(df_test.iloc[:,0]).reshape(-1,1))

_temp_test_input = np.hstack((x_test_input[:,:8],x_test_input[:,-3:-1]))
_temp_test_label = x_test_label
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

_temp_train_input = np.hstack((x_train_input[:, :8], x_train_input[:, -3:-1]))
_temp_train_label = x_train_label

# 将数据转换为 PyTorch 张量
train_input_tensor = torch.tensor(_temp_train_input.astype(np.float32))
train_label_tensor = torch.tensor(_temp_train_label.astype(np.float32))


In [2]:
# description source code annotation
# prune_node source code auto_save 

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from concurrent.futures import ThreadPoolExecutor

# 定义 k 折交叉验证
k = 3  # 例如，3折交叉验证
kf = KFold(n_splits=k, shuffle=True, random_state=42)

def evaluate_params(step, lr, grid, k, internal_units, batch, lamb, lamb_entropy, train_input_tensor, train_label_tensor, device):
    try:
        fold_losses = []
        for fold, (train_index, val_index) in enumerate(kf.split(train_input_tensor)):
            dataset = dict()
            # 获取训练和验证数据
            train_input_fold = train_input_tensor[train_index]
            train_label_fold = train_label_tensor[train_index]
            val_input_fold = train_input_tensor[val_index]
            val_label_fold = train_label_tensor[val_index]
            dataset['train_input'] = train_input_fold.clone().detach()
            dataset['train_label'] = train_label_fold.clone().detach()
            dataset['test_input'] = val_input_fold.clone().detach()
            dataset['test_label'] = val_label_fold.clone().detach()

            model = KAN(width=[10, internal_units, 1], grid=grid, k=k, auto_save=False, seed=0, device=device)
            for i in range(internal_units):
                model.fix_symbolic(0, 9, i, 'sin', fit_params_bool=False)
                model.fix_symbolic(0, 8, i, 'sin', fit_params_bool=False)
            try:
                model.fit(dataset, opt="Adam", steps=step, batch=batch, lr=lr, lamb=lamb, lamb_entropy=lamb_entropy)
                val_predictions = model.forward(dataset['test_input'])
                val_loss = mean_squared_error(dataset['test_label'].detach().numpy(), val_predictions.detach().numpy())
            except Exception as e:
                print(f"Error during model fitting: {e}")
                val_loss = 100

            fold_losses.append(val_loss)

        # 计算当前超参数组合在所有折上的平均损失
        avg_val_loss = sum(fold_losses) / len(fold_losses)
        return {
            'step': step,
            'lr': lr,
            'grid': grid,
            'k': k,
            'internal_units': internal_units,
            'batch': batch,
            'lamb': lamb, 
            'lamb_entropy': lamb_entropy,
            'avg_val_loss': avg_val_loss
        }
    except Exception as e:
        print(f"Error in evaluate_params: {e}")
        return None

# 用于存储每个折的性能
results_1 = []
for step in [50, 80, 100]:
    for lr in [0.01, 0.001, 0.0001]:
        for grid in [8,10,12]:
            for k in [2, 3]:
                for lamb in [0.01,0.1]:
                    for lamb_entropy in [10,100]:
                        for internal_units in [12, 16]:
                            for batch in [32, 64]:
                                results_1.append(evaluate_params(step, lr, grid, k, internal_units, batch, lamb, lamb_entropy, train_input_tensor, train_label_tensor, device))
                #                 break
                #             break
                #         break
                #     break
                # break
    
    print(f'{step} finish!')
    results_df = pd.DataFrame(results_1)
    results_df.to_csv('./parameters_configuration_1_'+str(step)+'_SPRING.csv')  
    # break

50 finish!
80 finish!
100 finish!


In [3]:
results_df.to_csv('./parameters_configuration/KAN_BaseConfig_SPRING.csv') 